# Report for models 

#### Import the needed libraries

In [ ]:
from face_alignment import FaceAlignment, LandmarksType

from giskard_vision.landmark_detection.dataloaders.loaders import DataLoader300W
from giskard_vision.landmark_detection.dataloaders.wrappers import (
    CroppedDataLoader,
    ResizedDataLoader,
    HeadPoseDataLoader,
    EthnicityDataLoader,
)
from giskard_vision.core.dataloaders.wrappers import (
    CachedDataLoader,
    FilteredDataLoader,
    ColoredDataLoader,
    BlurredDataLoader,
)

from giskard_vision.landmark_detection.models.wrappers import OpenCVWrapper, FaceAlignmentWrapper
from giskard_vision.landmark_detection.tests.performance import NMEMean
from giskard_vision.landmark_detection.marks.facial_parts import FacialParts
from giskard_vision.landmark_detection.tests.report import Report

#### Load the main dataset

In [2]:
dl_ref = DataLoader300W(dir_path="../datasets/300W/sample")

#### Define your criteria

In [3]:
# cropping
dl_cropped_left = CroppedDataLoader(dl_ref, part=FacialParts.LEFT_HALF.value)
dl_cropped_upper = CroppedDataLoader(dl_ref, part=FacialParts.UPPER_HALF.value)

# resizing
dl_resized = ResizedDataLoader(dl_ref, scales=0.5)

# coloring
dl_colored = ColoredDataLoader(dl_ref)

# blurring
dl_blurred = BlurredDataLoader(dl_ref)


# head pose filtering
def positive_roll(elt):
    return elt[2]["headPose"]["roll"] > 0


def negative_roll(elt):
    return elt[2]["headPose"]["roll"] < 0


cached_dl = CachedDataLoader(HeadPoseDataLoader(dl_ref), cache_size=None, cache_img=False, cache_marks=False)
dl_positive_roll = FilteredDataLoader(cached_dl, positive_roll)
dl_negative_roll = FilteredDataLoader(cached_dl, negative_roll)


# ethnicity filtering
def white_ethnicity(elt):
    return elt[2]["ethnicity"] == "white"


def latino_ethnicity(elt):
    return elt[2]["ethnicity"] == "latino hispanic"


cached_dl = CachedDataLoader(
    EthnicityDataLoader(dl_ref, ethnicity_map={"indian": "asian"}), cache_size=None, cache_img=False, cache_marks=False
)
dl_white = FilteredDataLoader(cached_dl, white_ethnicity)
dl_latino = FilteredDataLoader(cached_dl, latino_ethnicity)

dataloaders_list = [
    dl_cropped_left,
    dl_cropped_upper,
    dl_resized,
    dl_colored,
    dl_blurred,
    dl_positive_roll,
    dl_negative_roll,
    dl_white,
    dl_latino,
]

2024-01-17 12:01:44.698306: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### Initialise the models you'd like to compare

In [4]:
models_list = [
    FaceAlignmentWrapper(model=FaceAlignment(LandmarksType.TWO_D, device="cpu", flip_input=False)),
    OpenCVWrapper(),
]

models_list = [models_list[1]]

loading data from : lbfmodel.yaml


#### Create the report

In [5]:
report = Report(models_list, dataloaders_list, dataloader_ref=dl_ref)

OpenCVWrapper: Face not detected in processed image of batch 1 and index 0.
OpenCVWrapper: Face not detected in processed image of batch 5 and index 0.
OpenCVWrapper: Face not detected in processed image of batch 1 and index 0.
OpenCVWrapper: Face not detected in processed image of batch 2 and index 0.
OpenCVWrapper: Face not detected in processed image of batch 3 and index 0.
OpenCVWrapper: Face not detected in processed image of batch 5 and index 0.


#### Convert the report into a dataframe

In [6]:
report.to_dataframe()

,model,facial_part,dataloader,prediction_time,prediction_fail_rate,test,metric,metric_value,threshold,passed
0,OpenCV,left half,300W cropped on left half,0.929891,0.564706,TestDiff,NME_mean,-0.644057,-0.1,True
1,OpenCV,upper half,300W cropped on upper half,0.943296,0.682353,TestDiff,NME_mean,0.040216,-0.1,False
2,OpenCV,entire face,300W resizing with ratios: 0.5,0.995143,0.000000,TestDiff,NME_mean,-0.079876,-0.1,False
3,OpenCV,entire face,300W altered with color mode 7,1.316452,0.000000,TestDiff,NME_mean,0.001347,-0.1,False
4,OpenCV,entire face,300W blurred,1.483105,0.000000,TestDiff,NME_mean,-0.103017,-0.1,True
5,OpenCV,entire face,(Cached (300W) with head-pose) filtered using ...,0.974038,0.000000,TestDiff,NME_mean,0.077927,-0.1,False
6,OpenCV,entire face,(Cached (300W) with head-pose) filtered using ...,2.076661,0.000000,TestDiff,NME_mean,-0.019482,-0.1,False
7,OpenCV,entire face,(Cached (300W) with ethnicity) filtered using ...,2.389328,0.000000,TestDiff,NME_mean,0.168421,-0.1,False
8,OpenCV,entire face,(Cached (300W) with ethnicity) filtered using ...,1.945159,0.000000,TestDiff,NME_mean,-0.784538,-0.1,True


#### Full Report
Here's a full prepared report to compare `FaceAlignment` and `OpenCV` models on the full `300W-indoor` dataset.

In [1]:
import pandas as pd

full_report = pd.read_csv("full_report.csv")
full_report

,model,facial_part,dataloader,prediction_time,prediction_fail_rate,test,metric,metric_value,threshold,passed
0,FaceAlignment,left half,300W cropped on left half,97.645192,0.968260,TestDiff,NME_mean,-0.627014,-0.1,True
1,FaceAlignment,upper half,300W cropped on upper half,77.467558,0.971765,TestDiff,NME_mean,-0.587295,-0.1,True
2,FaceAlignment,entire face,300W resizing with ratios: 0.5,123.234188,0.733333,TestDiff,NME_mean,0.691124,-0.1,False
3,FaceAlignment,entire face,300W altered with color mode 7,77.377963,0.943333,TestDiff,NME_mean,0.152823,-0.1,False
4,FaceAlignment,entire face,300W blurred,78.804337,0.943333,TestDiff,NME_mean,0.448503,-0.1,False
5,FaceAlignment,entire face,(Cached (300W) with head-pose) filtered using ...,67.365618,0.949444,TestDiff,NME_mean,-0.611439,-0.1,True
6,FaceAlignment,entire face,(Cached (300W) with head-pose) filtered using ...,49.014326,0.934167,TestDiff,NME_mean,1.990828,-0.1,False
7,FaceAlignment,entire face,(Cached (300W) with ethnicity) filtered using ...,52.884618,0.950238,TestDiff,NME_mean,-0.695888,-0.1,True
8,FaceAlignment,entire face,(Cached (300W) with ethnicity) filtered using ...,40.927982,0.892719,TestDiff,NME_mean,3.921029,-0.1,False
9,OpenCV,left half,300W cropped on left half,318.501807,0.659020,TestDiff,NME_mean,-0.944248,-0.1,True
